In [1]:
!pwd

/home/kji/workspace/jupyter_kji/DeepLearningWithNumpy


In [2]:
!git clone https://github.com/kimjiil/DeepLearningWithNumpy.git

fatal: 대상 경로가('DeepLearningWithNumpy') 이미 있고 빈 디렉터리가 아닙니다.


In [3]:
!ls

 DeepLearningWithNumpy	'myDeepLearning Code.ipynb'


In [4]:
%cd DeepLearningWithNumpy/

/home/kji/workspace/jupyter_kji/DeepLearningWithNumpy/DeepLearningWithNumpy


In [5]:
!pwd

/home/kji/workspace/jupyter_kji/DeepLearningWithNumpy/DeepLearningWithNumpy


In [ ]:
!conda install -c conda-forge cupy -y

In [6]:
from myLib.Module import myModule, mySequential, myTensor
from myLib.Layer import *
from myLib.LossFunc import *
from myLib.Optimizer import *
import myLib

from torchvision.datasets import MNIST
import numpy as np
import cupy as cp

## SImple CNN Model

In [7]:
class my_model(myModule):
    def __init__(self):
        super(my_model, self).__init__()

        self.convlayers = mySequential(
            Conv2d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=2, bias=False),
            ReLU(),
            MaxPool2d(kernel_size=2, stride=2, padding=0),
            Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=0, bias=False),
            ReLU(),
            MaxPool2d(kernel_size=2, stride=2, padding=0)
        )
        self.flatten = Flatten()
        self.hidden_layers = mySequential(
            Linear(in_features=1152, out_features=256, bias=True),
            ReLU(),
        )
        self.classifier = Linear(in_features=256, out_features=10, bias=True)
        self.sigmoid = Sigmoid()


    def forward(self, x):
        x = self.convlayers(x)
        x = self.flatten(x)
        x = self.hidden_layers(x)
        x = self.classifier(x)
        x = self.sigmoid(x)
        return x

In [8]:
import tqdm

download_path = "./MNIST_Datset"
train_dataset = MNIST(download_path, train=True, download=True)
valid_dataset = MNIST(download_path, train=False, download=True)

/home/kji/anaconda3/envs/py39_0/lib/python3.9/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448255797/work/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [9]:
model = my_model()
model.to(device="cuda:0")

optimizer = Adam(model.parameters(), lr=0.001)
criterion = MSELoss()
epoch_size = 10
batch_size = 144
total_size = len(train_dataset)
for epoch_i in range(epoch_size):
    start_time = time.time()
    step_size = int(total_size / batch_size)
    loss_sum = []
    for step_i in tqdm.tqdm(range(step_size)):
        input_data = train_dataset.data[step_i * batch_size:(step_i + 1) * batch_size].numpy().reshape(batch_size,1, 28, 28) /255.
        targets = train_dataset.targets[step_i * batch_size:(step_i + 1) * batch_size].numpy()
        targets_one_hot = np.eye(10)[targets]

        input_data = myTensor(input_data).to(device="cuda:0")
        targets = myTensor(targets_one_hot).to(device="cuda:0")

        optimizer.zero_grad()
        pred = model(input_data)

        loss = criterion(pred, targets)
        loss.backward()
        optimizer.step()

        # print(step_i, loss)
        loss_sum.append(loss.data)
    end_time = time.time()
    print(epoch_i, "loss", sum(loss_sum) / len(loss_sum), "runtime", round(end_time - start_time, 5), 's')

    step_size = int(len(valid_dataset.data) / batch_size)

    acc_list = []
    for step_i in tqdm.tqdm(range(step_size)):
        input_data = valid_dataset.data[step_i * batch_size:(step_i + 1) * batch_size].numpy().reshape(batch_size,1, 28, 28) / 255.
        targets = valid_dataset.targets[step_i * batch_size:(step_i + 1) * batch_size].numpy()

        input_data = myTensor(input_data).to(device="cuda:0")
        targets = myTensor(targets).to(device="cuda:0")
        pred = model(input_data)
        pred_label = myLib.argmax(pred, 1)

        correct = (targets == pred_label).numpy()
        acc_list.extend(correct)

    print(f"accuracy {np.mean(acc_list)}")

100%|████████████████████████████████████████████████████████████████████████████████████████| 416/416 [04:26<00:00,  1.56it/s]


0 loss 48.42356527823377 runtime 266.65035 s


100%|██████████████████████████████████████████████████████████████████████████████████████████| 69/69 [00:14<00:00,  4.74it/s]


accuracy 0.6395933977455717


100%|████████████████████████████████████████████████████████████████████████████████████████| 416/416 [04:20<00:00,  1.59it/s]


...

accuracy 0.9899355877616747


100%|████████████████████████████████████████████████████████████████████████████████████████| 416/416 [03:58<00:00,  1.75it/s]


9 loss 0.6647467722348354 runtime 238.1049 s


100%|██████████████████████████████████████████████████████████████████████████████████████████| 69/69 [00:13<00:00,  4.95it/s]

accuracy 0.9882246376811594
